Compare the distribution of Airbnbs and other traditional accommodation types such as hotels.

data source: https://data.cityofnewyork.us/City-Government/Hotels-Properties-Citywide/tjus-cn27

In [38]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
%matplotlib inline

In [39]:
df_hotel = pd.read_csv('../data/Hotels_Properties_Citywide.csv')

In [40]:
df_hotel.columns

Index(['PARID', 'BOROCODE', 'BLOCK', 'LOT', 'TAXYEAR', 'STREET NUMBER',
       'STREET NAME', 'Postcode', 'BLDG_CLASS', 'TAXCLASS', 'OWNER_NAME',
       'Borough', 'Latitude', 'Longitude', 'Community Board',
       'Council District', 'Census Tract', 'BIN', 'BBL', 'NTA'],
      dtype='object')

Description for columns:  

'PARID': No description, Doesn't seem important  

'BOROCODE': Don't need it because of 'Borough'  

'BLOCK': Borough, Block, and Lot (BBL) is the parcel number system used to identify each unit of real estate in New York City for numerous city purposes. It consists of three numbers, separated by slashes; the borough, which is 1 digit; the block number, which is up to 5 digits; and the lot number, which is up to 4 digits.  

'LOT'  

'TAXYEAR': An annual accounting period for keeping records and reporting income and expenses. We're not investigating tax, so don't need it

'STREET NUMBER'
'STREET NAME'
'Postcode'

'BLDG_CLASS': Building class (Use and Occupancy classification: https://igpny.com/wp-content/uploads/2019/05/NYC-DOB-Building-Code-Chapter-3-Use-and-Occupancy-Classification.pdf). I don't think we need it.

'TAXCLASS': We're not interested in tax here

'OWNER_NAME': Do we wanna check if owner of Airbnb and hotels is the same?

'Borough': We need it

'Latitude': We need it

'Longitude': We need it

'Community Board': Membership - Community Boards are local representative bodies. There are 59 throughout the city. Each Board consists of up to 50 unsalaried members appointed by the Borough President, with half nominated by the City Council Members who represent the community district.
Are we interested in if airbnbs are nearby the community board? 

'Council District': Council District means any of four political subdivisions within the City by which City Council members are elected.
Are we interested in if airbnbs are nearby the community board? 

'Census Tract',
'BIN': Building Identification Number. Don't think we need this.

'BBL': Borough, Block, Lot

'NTA': Neighborhood Tabulation Areas; created by the NYC Dept of Planning by aggregating census tracts into 195 neighborhood-like areas.
Maybe interesting? Because it's neighborhood like areas.


Questions: Do we need BBL, Street number/name, Postcode?  
I think only for a geographic use, borough, latitue, longitude are enough? 

In [41]:
df_hotel.head()

,PARID,BOROCODE,BLOCK,LOT,TAXYEAR,STREET NUMBER,STREET NAME,Postcode,BLDG_CLASS,TAXCLASS,OWNER_NAME,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,1000080039,1,8,39,2021,32,PEARL STREET,10004,H3,4,"32 PEARL, LLC",MANHATTAN,40.703235,-74.012421,101.0,1.0,9.0,1078968.0,1.000080e+09,Battery Park City-Lower Manhattan
1,1000080051,1,8,51,2021,6,WATER STREET,10004,H2,4,AI IV LLC,MANHATTAN,40.702744,-74.012201,101.0,1.0,9.0,1090472.0,1.000080e+09,Battery Park City-Lower Manhattan
2,1000100033,1,10,33,2021,8,STONE STREET,10004,H2,4,"B.H. 8 STONE STREET AG, LLC",MANHATTAN,40.704025,-74.012638,101.0,1.0,9.0,1087618.0,1.000100e+09,Battery Park City-Lower Manhattan
3,1000110029,1,11,29,2021,11,STONE STREET,10004,H2,4,"PREMIER EMERALD, LLC",MANHATTAN,40.704039,-74.012317,101.0,1.0,9.0,1000041.0,1.000110e+09,Battery Park City-Lower Manhattan
4,1000161301,1,16,1301,2021,102,NORTH END AVENUE,10282,RH,4,GOLDMAN SACHS,MANHATTAN,40.714812,-74.016153,101.0,1.0,31703.0,1085867.0,1.000168e+09,Battery Park City-Lower Manhattan


In [42]:
# check the data size
print(df_hotel.info())
print(df_hotel.describe())
print('Data`s Shape: ', df_hotel.shape)
print('\nType of features \n', df_hotel.dtypes.value_counts())
isnull_series = df_hotel.isnull().sum()
isna_series = df_hotel.isna().sum()
print('\nNull columns and numbers:\n ', isnull_series[isnull_series > 0].sort_values(ascending=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5519 entries, 0 to 5518
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PARID             5519 non-null   int64  
 1   BOROCODE          5519 non-null   int64  
 2   BLOCK             5519 non-null   int64  
 3   LOT               5519 non-null   int64  
 4   TAXYEAR           5519 non-null   int64  
 5   STREET NUMBER     5514 non-null   object 
 6   STREET NAME       5519 non-null   object 
 7   Postcode          5519 non-null   int64  
 8   BLDG_CLASS        5519 non-null   object 
 9   TAXCLASS          5519 non-null   int64  
 10  OWNER_NAME        5519 non-null   object 
 11  Borough           5514 non-null   object 
 12  Latitude          5502 non-null   float64
 13  Longitude         5502 non-null   float64
 14  Community Board   5502 non-null   float64
 15  Council District  5502 non-null   float64
 16  Census Tract      5502 non-null   float64


In [43]:
df_hotel.value_counts(['Borough']).sort_index()

Borough  
1            2284
2              78
3             187
4             223
5              16
BRONX          73
BROOKLYN      177
MANHATTAN    2250
QUEENS        209
STATEN IS      17
dtype: int64

In [44]:
df_hotel.value_counts(['NTA']).sort_index()

NTA                                       
Allerton-Pelham Gardens                         2
Annadale-Huguenot-Prince's Bay-Eltingville      2
Astoria                                         9
Auburndale                                      4
Baisley Park                                   15
                                             ... 
Williamsbridge-Olinville                        4
Williamsburg                                    2
Woodlawn-Wakefield                              9
Woodside                                        8
Yorkville                                     220
Length: 136, dtype: int64

In the end we're just using borough, latitue, longitude



In [45]:
# drop all rows with any NaN and NaT values
df_hotel_clean = df_hotel.dropna(inplace=False)

In [46]:
df_hotel_clean.head()

,PARID,BOROCODE,BLOCK,LOT,TAXYEAR,STREET NUMBER,STREET NAME,Postcode,BLDG_CLASS,TAXCLASS,OWNER_NAME,Borough,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,1000080039,1,8,39,2021,32,PEARL STREET,10004,H3,4,"32 PEARL, LLC",MANHATTAN,40.703235,-74.012421,101.0,1.0,9.0,1078968.0,1.000080e+09,Battery Park City-Lower Manhattan
1,1000080051,1,8,51,2021,6,WATER STREET,10004,H2,4,AI IV LLC,MANHATTAN,40.702744,-74.012201,101.0,1.0,9.0,1090472.0,1.000080e+09,Battery Park City-Lower Manhattan
2,1000100033,1,10,33,2021,8,STONE STREET,10004,H2,4,"B.H. 8 STONE STREET AG, LLC",MANHATTAN,40.704025,-74.012638,101.0,1.0,9.0,1087618.0,1.000100e+09,Battery Park City-Lower Manhattan
3,1000110029,1,11,29,2021,11,STONE STREET,10004,H2,4,"PREMIER EMERALD, LLC",MANHATTAN,40.704039,-74.012317,101.0,1.0,9.0,1000041.0,1.000110e+09,Battery Park City-Lower Manhattan
4,1000161301,1,16,1301,2021,102,NORTH END AVENUE,10282,RH,4,GOLDMAN SACHS,MANHATTAN,40.714812,-74.016153,101.0,1.0,31703.0,1085867.0,1.000168e+09,Battery Park City-Lower Manhattan


In [47]:
isnull_series = df_hotel_clean.isnull().sum()
isna_series = df_hotel_clean.isna().sum()
print(isnull_series, isna_series)

PARID               0
BOROCODE            0
BLOCK               0
LOT                 0
TAXYEAR             0
STREET NUMBER       0
STREET NAME         0
Postcode            0
BLDG_CLASS          0
TAXCLASS            0
OWNER_NAME          0
Borough             0
Latitude            0
Longitude           0
Community Board     0
Council District    0
Census Tract        0
BIN                 0
BBL                 0
NTA                 0
dtype: int64 PARID               0
BOROCODE            0
BLOCK               0
LOT                 0
TAXYEAR             0
STREET NUMBER       0
STREET NAME         0
Postcode            0
BLDG_CLASS          0
TAXCLASS            0
OWNER_NAME          0
Borough             0
Latitude            0
Longitude           0
Community Board     0
Council District    0
Census Tract        0
BIN                 0
BBL                 0
NTA                 0
dtype: int64


In [48]:
# remove duplicates of the same ID 
# what if there are different rooms in the same hotel building? For now I assume we keep them
# df_hotel_clean.drop_duplicates(subset='PARID', keep="last")

In [49]:
df_hotel_clean_2 = df_hotel_clean[["PARID", "Borough", "Postcode", "NTA"]]
df_hotel_clean_2.head()

,PARID,Borough,Postcode,NTA
0,1000080039,MANHATTAN,10004,Battery Park City-Lower Manhattan
1,1000080051,MANHATTAN,10004,Battery Park City-Lower Manhattan
2,1000100033,MANHATTAN,10004,Battery Park City-Lower Manhattan
3,1000110029,MANHATTAN,10004,Battery Park City-Lower Manhattan
4,1000161301,MANHATTAN,10282,Battery Park City-Lower Manhattan


In [50]:
print(df_hotel_clean_2.info())
print(df_hotel_clean_2.describe())
print('Data`s Shape: ', df_hotel_clean_2.shape)
print('\nType of features \n', df_hotel_clean_2.dtypes.value_counts())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5473 entries, 0 to 5517
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PARID     5473 non-null   int64 
 1   Borough   5473 non-null   object
 2   Postcode  5473 non-null   int64 
 3   NTA       5473 non-null   object
dtypes: int64(2), object(2)
memory usage: 213.8+ KB
None
              PARID      Postcode
count  5.473000e+03   5473.000000
mean   1.421075e+09  10211.873744
std    9.567826e+08    436.099099
min    1.000080e+09  10001.000000
25%    1.008370e+09  10016.000000
50%    1.011381e+09  10019.000000
75%    1.013370e+09  10036.000000
max    5.073650e+09  11694.000000
Data`s Shape:  (5473, 4)

Type of features 
 int64     2
object    2
dtype: int64


In [51]:
df_hotel_clean_2['Borough'] = df_hotel_clean_2['Borough'].replace(['1', '2', '3', '4', '5'], 
['Manhattan', 'Bronx', 'Brooklyn', 'Queens', 'Staten Island'])
df_hotel_clean_2.head()

,PARID,Borough,Postcode,NTA
0,1000080039,MANHATTAN,10004,Battery Park City-Lower Manhattan
1,1000080051,MANHATTAN,10004,Battery Park City-Lower Manhattan
2,1000100033,MANHATTAN,10004,Battery Park City-Lower Manhattan
3,1000110029,MANHATTAN,10004,Battery Park City-Lower Manhattan
4,1000161301,MANHATTAN,10282,Battery Park City-Lower Manhattan


In [52]:
df_hotel_clean_2['Borough'] = df_hotel_clean_2['Borough'].replace(['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN IS'], 
['Manhattan', 'Bronx', 'Brooklyn', 'Queens', 'Staten Island'])
df_hotel_clean_2.head()
# df_hotel_clean_2.to_csv('../data/Hotels_clean.csv')

,PARID,Borough,Postcode,NTA
0,1000080039,Manhattan,10004,Battery Park City-Lower Manhattan
1,1000080051,Manhattan,10004,Battery Park City-Lower Manhattan
2,1000100033,Manhattan,10004,Battery Park City-Lower Manhattan
3,1000110029,Manhattan,10004,Battery Park City-Lower Manhattan
4,1000161301,Manhattan,10282,Battery Park City-Lower Manhattan


In [53]:
df_hotel_clean_2.rename(columns={'Postcode': 'zipcode', 'Borough': 'neighbourhood_group', 'NTA': 'neighbourhood'}, inplace=True)
df_hotel_clean_2.head()

,PARID,neighbourhood_group,zipcode,neighbourhood
0,1000080039,Manhattan,10004,Battery Park City-Lower Manhattan
1,1000080051,Manhattan,10004,Battery Park City-Lower Manhattan
2,1000100033,Manhattan,10004,Battery Park City-Lower Manhattan
3,1000110029,Manhattan,10004,Battery Park City-Lower Manhattan
4,1000161301,Manhattan,10282,Battery Park City-Lower Manhattan


In [54]:
df_hotel_clean_2['neighbourhood'] = df_hotel_clean_2['neighbourhood'].str.replace('-', ' ')
df_hotel_clean_2.head()

,PARID,neighbourhood_group,zipcode,neighbourhood
0,1000080039,Manhattan,10004,Battery Park City Lower Manhattan
1,1000080051,Manhattan,10004,Battery Park City Lower Manhattan
2,1000100033,Manhattan,10004,Battery Park City Lower Manhattan
3,1000110029,Manhattan,10004,Battery Park City Lower Manhattan
4,1000161301,Manhattan,10282,Battery Park City Lower Manhattan


In [55]:
df_zipcode = pd.read_csv('../data/neighbourhoods.csv')

In [68]:
df_zipcode.head()

,neighbourhood_group,neighbourhood,zipcode
0,Bronx,Allerton,10467
1,Bronx,Baychester,10469
2,Bronx,Belmont,10457
3,Bronx,Belmont,10458
4,Bronx,Bronx Park,10460


In [56]:
df_hotel_clean_2.shape

(5473, 4)

In [57]:
for value in df_zipcode['neighbourhood'].unique():
    df_hotel_clean_2['neighbourhood'] = df_hotel_clean_2['neighbourhood'].replace(to_replace=r'^{}.*'.format(value), value=value, regex=True)

df_hotel_clean_2.head()

,PARID,neighbourhood_group,zipcode,neighbourhood
0,1000080039,Manhattan,10004,Battery Park City
1,1000080051,Manhattan,10004,Battery Park City
2,1000100033,Manhattan,10004,Battery Park City
3,1000110029,Manhattan,10004,Battery Park City
4,1000161301,Manhattan,10282,Battery Park City


In [58]:
df_hotel_clean_2['hotel_counts_per_neighbourhood'] = df_hotel_clean_2.groupby('neighbourhood')['neighbourhood'].transform('count')

In [59]:
df_hotel_clean_2.head()

,PARID,neighbourhood_group,zipcode,neighbourhood,hotel_counts_per_neighbourhood
0,1000080039,Manhattan,10004,Battery Park City,95
1,1000080051,Manhattan,10004,Battery Park City,95
2,1000100033,Manhattan,10004,Battery Park City,95
3,1000110029,Manhattan,10004,Battery Park City,95
4,1000161301,Manhattan,10282,Battery Park City,95


In [64]:
# show me hotel_counts_per_neighbourhood in each neighbourhood
df_hotel_clean_2.value_counts(['neighbourhood', 'hotel_counts_per_neighbourhood']).sort_index()

neighbourhood                               hotel_counts_per_neighbourhood
Allerton                                    2                                   2
Annadale Huguenot Prince's Bay Eltingville  2                                   2
Astoria                                     9                                   9
Auburndale                                  4                                   4
Baisley Park                                15                                 15
                                                                             ... 
Williamsbridge                              4                                   4
Williamsburg                                2                                   2
Woodlawn                                    9                                   9
Woodside                                    8                                   8
Yorkville                                   220                               220
Length: 126, dtype: int

airbnb

In [88]:
df_airbnb = pd.read_csv('../data/airbnb_open_data_full_clean.csv')
df_airbnb['airbnb_counts_per_neighbourhood'] = df_airbnb.groupby('neighbourhood')['neighbourhood'].transform('count')
df_airbnb.head()

,id,name,host_id,host_name,neighbourhood,neighbourhood_group,latitude,longitude,price,calculated_host_listings_count,host_listings_neighbourhood_count,host_listings_neighbourhood_group_count,has_fire_alarm,has_co_monitor,airbnb_counts_per_neighbourhood
0,2539,Clean & quiet apt home by the park,2787,John,Kensington,Brooklyn,40.64529,-73.97238,299.0,9,2,9,True,True,117
1,2595,Skylit Midtown Castle,2845,Jennifer,Midtown,Manhattan,40.75356,-73.98559,175.0,3,1,3,True,True,1701
2,5121,BlissArtsSpace!,7356,Garon,Bedford-Stuyvesant,Brooklyn,40.68535,-73.95512,60.0,2,2,2,False,False,2779
3,45910,Beautiful Queens Brownstone! - 5BR,204539,Mark,Ridgewood,Queens,40.70309,-73.89963,425.0,6,3,6,True,True,315
4,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Sunset Park,Brooklyn,40.66265,-73.99454,275.0,1,1,1,True,True,297


In [66]:
df_hotel_temp = df_hotel_clean_2[['neighbourhood', 'neighbourhood_group', 'zipcode', 'hotel_counts_per_neighbourhood']].copy()
df_hotel_temp.drop_duplicates(subset='neighbourhood', keep="last", inplace=True)
df_hotel_temp.reset_index(drop=True, inplace=True)
df_hotel_temp.head()

,neighbourhood,neighbourhood_group,zipcode,hotel_counts_per_neighbourhood
0,Battery Park City,Manhattan,10038,95
1,Lower East Side,Manhattan,10002,2
2,Chinatown,Manhattan,10002,73
3,East Village,Manhattan,10003,30
4,SoHo,Manhattan,10013,854


In [85]:
#df_airbnb_zip = df_airbnb.merge(df_zipcode[['neighbourhood', 'neighbourhood_group', 'zipcode']], on=['neighbourhood', 'neighbourhood_group'])
#df_airbnb_zip.head()

,id,name,host_id,host_name,neighbourhood,neighbourhood_group,latitude,longitude,price,calculated_host_listings_count,host_listings_neighbourhood_count,host_listings_neighbourhood_group_count,has_fire_alarm,has_co_monitor,airbnb_counts_per_neighbourhood,zipcode
0,2539,Clean & quiet apt home by the park,2787,John,Kensington,Brooklyn,40.645290,-73.972380,299.0,9,2,9,True,True,117,11218
1,267652,Private clean spacious room - Across from subway,164675,Janice,Kensington,Brooklyn,40.642954,-73.972928,75.0,1,1,1,True,True,117,11218
2,452068,"Spacious 4 bedroom house, New York",2246071,Tamara,Kensington,Brooklyn,40.643210,-73.971370,200.0,1,1,1,False,False,117,11218
3,656281,"QUIET, SPACIOUS, COMFORTABLE, & GREAT LOCATION",3180741,Manon,Kensington,Brooklyn,40.646500,-73.979590,83.0,1,1,1,True,True,117,11218
4,848170,Gorgeous 1BR near Prospect Park,4428278,Rohit,Kensington,Brooklyn,40.646260,-73.975970,100.0,1,1,1,True,True,117,11218


In [89]:
df_airbnb_temp = df_airbnb[['neighbourhood', 'neighbourhood_group', 'airbnb_counts_per_neighbourhood']].copy()
df_airbnb_temp.drop_duplicates(subset='neighbourhood', keep="last", inplace=True)
df_airbnb_temp.reset_index(drop=True, inplace=True)
df_airbnb_temp.head()

,neighbourhood,neighbourhood_group,airbnb_counts_per_neighbourhood
0,Fort Wadsworth,Staten Island,1
1,New Dorp,Staten Island,1
2,Neponsit,Queens,2
3,Country Club,Bronx,1
4,Willowbrook,Staten Island,1


In [90]:
df_airbnb_temp.value_counts(['neighbourhood', 'airbnb_counts_per_neighbourhood']).sort_index()

neighbourhood    airbnb_counts_per_neighbourhood
Allerton         45                                 1
Arden Heights    7                                  1
Arrochar         17                                 1
Arverne          110                                1
Astoria          686                                1
                                                   ..
Windsor Terrace  114                                1
Woodhaven        85                                 1
Woodlawn         7                                  1
Woodrow          1                                  1
Woodside         336                                1
Length: 243, dtype: int64

In [91]:
df_airbnb_hotel = df_airbnb_temp.merge(df_hotel_temp, on=['neighbourhood', 'neighbourhood_group'])
df_airbnb_hotel.head()

,neighbourhood,neighbourhood_group,airbnb_counts_per_neighbourhood,zipcode,hotel_counts_per_neighbourhood
0,New Dorp,Staten Island,1,10306,2
1,Westerleigh,Staten Island,1,10314,2
2,New Springville,Staten Island,3,10314,10
3,Grymes Hill,Staten Island,7,10301,2
4,Woodlawn,Bronx,7,10470,9


In [96]:
df_airbnb_hotel.to_csv('../data/Hotels_Airbnbs_Neighbourhood_counts.csv')